#### DATA 만들기 

In [1]:
import os
import json
import pandas as pd # Gzip 압축된 CSV 파일 불러오기

df = pd.read_csv("data/data.csv.gz", compression='gzip')
df.head()

,standard_form,dialect_form
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도
1,어어 그니까,어어 그니까
2,아 잘도,아 잘도
3,예초기 뱀도 짤려,예초기 뱀도 짤려
4,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해


In [2]:
# 중복 제거
df_filtered = df[df["standard_form"] != df["dialect_form"]]
df_filtered = df_filtered.drop_duplicates()
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116389 entries, 6 to 2774254
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   standard_form  1116389 non-null  object
 1   dialect_form   1116389 non-null  object
dtypes: object(2)
memory usage: 25.6+ MB


In [3]:
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.head()

,standard_form,dialect_form
0,예전에 그렇게,예전에 겅
1,아 그러니까 예초 할땐 조심해야 돼,아 겅하니까이 예초 할땐 조심해야 돼
2,그러고 예초 벌초(하다그네)/(하다가),겅허고 예초 벌초(하다그네)/(하다가)
3,어 그거 튀겨서,어 그거 튀겨그네
4,돌멩이 해 가지고 실명된사람 있어,돌멩이 해 가지고 실명된사람 이서


In [5]:
# 간단한 전처리 
import re 
SPECIALS = "".join([".", ",", ";", ":", "!", "?", '"', "'", "&", "#", "$", "%"])

def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(r"[^가-힣\s]+", "", text)
    else:
        text = re.sub(rf"[^가-힣ㄱ-ㅎ0-9{SPECIALS}a-zA-Z\s]+", "", text)

    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()


df_filtered['dialect_form'] = df_filtered['dialect_form'].apply(preprocess)
df_filtered['standard_form'] = df_filtered['standard_form'].apply(preprocess)


In [6]:
### 80:20으로 train:test 나누기
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_filtered, test_size=0.2, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)


train, val = train_test_split(train, test_size=0.2, random_state=42)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

print(train.shape, val.shape, test.shape)

(714488, 2) (178623, 2) (223278, 2)


In [7]:
train.to_csv("data/train.csv", index=False)
val.to_csv("data/val.csv", index=False)
test.to_csv("data/test.csv", index=False)